In [9]:
import sys 
import subprocess
import os

In [10]:
resolution = "30"

In [11]:
folder_path = f"/home/data/national_fema_floodmap/rasters/{resolution}m"
if not os.path.exists(folder_path):
    os.mkdir(folder_path)

In [12]:
raster100y =  f"{folder_path}/national_fema_raster_100y_{resolution}m.tiff"
raster500y =  f"{folder_path}/national_fema_raster_500y_{resolution}m.tiff"
raster_combined = f"{folder_path}/national_fema_raster_combined_{resolution}m.tiff"

In [13]:
# Define the command as a list of arguments
command = [
    "gdal_rasterize",
    "-tr", resolution, resolution,
    "-sql", "SELECT * FROM USA_Flood_Hazard_Reduced_Set WHERE esri_symbology = '1% Annual Chance Flood Hazard'",
    "-burn", "1",
    "-ot", "Byte",
    "-co", "COMPRESS=LZW",
    "/home/data/national_fema_floodmap/efa6409b-9dab-4b36-b0aa-8641f1b26b6e.gdb",
    raster100y
]


# Execute the command
subprocess.run(command, check=True)

ERROR 1: Error occurred in /home/conda/feedstock_root/build_artifacts/gdal-split_1704802395302/work/ogr/ogrsf_frmts/openfilegdb/filegdbindex.cpp at line 800
ERROR 1: Error occurred in /home/conda/feedstock_root/build_artifacts/gdal-split_1704802395302/work/ogr/ogrsf_frmts/openfilegdb/filegdbindex.cpp at line 800
Warning 1: organizePolygons() received a polygon with more than 100 parts. The processing may be really slow.  You can skip the processing by setting METHOD=SKIP, or only make it analyze counter-clock wise parts by setting METHOD=ONLY_CCW if you can assume that the outline of holes is counter-clock wise defined


0...10...20...30...40...50...60...70...80...90...100 - done.


CompletedProcess(args=['gdal_rasterize', '-tr', '30', '30', '-sql', "SELECT * FROM USA_Flood_Hazard_Reduced_Set WHERE esri_symbology = '1% Annual Chance Flood Hazard'", '-burn', '1', '-ot', 'Byte', '-co', 'COMPRESS=LZW', '/home/data/national_fema_floodmap/efa6409b-9dab-4b36-b0aa-8641f1b26b6e.gdb', '/home/data/national_fema_floodmap/rasters/30m/national_fema_raster_100y_30m.tiff'], returncode=0)

In [14]:
import subprocess

# Define the command as a list of arguments
command = [
    "gdal_rasterize",
    "-tr", resolution, resolution,
    "-sql", "SELECT * FROM USA_Flood_Hazard_Reduced_Set WHERE esri_symbology = '0.2% Annual Chance Flood Hazard'",
    "-burn", "1",
    "-ot", "Byte",
    "-co", "COMPRESS=LZW",
    "/home/data/national_fema_floodmap/efa6409b-9dab-4b36-b0aa-8641f1b26b6e.gdb",
    raster500y
]


# Execute the command
subprocess.run(command, check=True)

ERROR 1: Error occurred in /home/conda/feedstock_root/build_artifacts/gdal-split_1704802395302/work/ogr/ogrsf_frmts/openfilegdb/filegdbindex.cpp at line 800
ERROR 1: Error occurred in /home/conda/feedstock_root/build_artifacts/gdal-split_1704802395302/work/ogr/ogrsf_frmts/openfilegdb/filegdbindex.cpp at line 800
Warning 1: organizePolygons() received a polygon with more than 100 parts. The processing may be really slow.  You can skip the processing by setting METHOD=SKIP, or only make it analyze counter-clock wise parts by setting METHOD=ONLY_CCW if you can assume that the outline of holes is counter-clock wise defined


0...10...20...30...40...50...60...70...80...90...100 - done.


CompletedProcess(args=['gdal_rasterize', '-tr', '30', '30', '-sql', "SELECT * FROM USA_Flood_Hazard_Reduced_Set WHERE esri_symbology = '0.2% Annual Chance Flood Hazard'", '-burn', '1', '-ot', 'Byte', '-co', 'COMPRESS=LZW', '/home/data/national_fema_floodmap/efa6409b-9dab-4b36-b0aa-8641f1b26b6e.gdb', '/home/data/national_fema_floodmap/rasters/30m/national_fema_raster_500y_30m.tiff'], returncode=0)

In [19]:
# Construct the command using sys.executable to ensure the correct Python interpreter is used
command = [
    sys.executable, "-m", "osgeo_utils.gdal_calc",
    "-A", raster100y,
    "-B", raster500y,
    "--outfile=" + raster_combined,
    "--calc=\"(A>0)*1 + (B>0)*(A<=0)*2\"",  # Conditional calculation
    "--NoDataValue=0",
    "--config", "CHECK_DISK_FREE_SPACE", "FALSE",
    "--co", "COMPRESS=LZW",
    "--overwrite"
]# Execute the command
subprocess.run(command, check=True)

/home/canyon/miniconda3/envs/myenv/lib/python3.9/site-packages/osgeo/gdal.py:312: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


0...10...20...30...40...50...60...70...80...90...100 - done.


CompletedProcess(args=['/home/canyon/miniconda3/envs/myenv/bin/python', '-m', 'osgeo_utils.gdal_calc', '-A', '/home/data/national_fema_floodmap/rasters/30m/national_fema_raster_100y_30m.tiff', '-B', '/home/data/national_fema_floodmap/rasters/30m/national_fema_raster_500y_30m.tiff', '--outfile=/home/data/national_fema_floodmap/rasters/30m/national_fema_raster_combined_30m.tiff', '--calc="(A>0)*1 + (B>0)*(A<=0)*2"', '--NoDataValue=0', '--config', 'CHECK_DISK_FREE_SPACE', 'FALSE', '--co', 'COMPRESS=LZW', '--overwrite'], returncode=0)

In [16]:
boundary_path = "/home/data/national_fema_floodmap/nola_testing_file.geojson" # Test paths
clipped_raster_path = f"{folder_path}/national_fema_raster_combined_clipped.tiff"

In [18]:
raster_combined

'/home/data/national_fema_floodmap/rasters/30m/national_fema_raster_combined_30m.tiff'

In [17]:
command = [
    "gdalwarp", "-cutline",
    boundary_path, "-crop_to_cutline",
    "-dstalpha", raster_combined,
    clipped_raster_path
]

# Execute the command
#subprocess.run(command, check=True)